<a href="https://colab.research.google.com/github/sivasamy1/data_analytics/blob/main/Parsing_JSON_and_persist_as_a_CSV_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pandas as pd
from tabulate import tabulate
import ast

In [ ]:
def pty_print(df):
  print(tabulate(df, headers='keys', tablefmt='github'))

In [32]:
url = 'https://raw.githubusercontent.com/leninworld/parser_exercises/refs/heads/main/json/students/student_marks.json'
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
else:
    print(f"Failed to fetch data: {response.status_code}")
    exit()
df = pd.json_normalize(data)
pty_print(df)

|    | name   |   age | marks                                                            | address                                                                                           | city      | gender   |   salary |
|----|--------|-------|------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|-----------|----------|----------|
|  0 | Raj    |    21 | [{'english': 91}, {'tamil': 99}, {'math': 87}, {'science': 89}]  | [{'20220101': '331 main road, chennai 600021'}, {'20210101': '33 gandhi road, bangalore 600021'}] | chennai   | male     |   200000 |
|  1 | Vim    |    28 | [{'english': 93}, {'tamil': 98}, {'math': 99}, {'science': 100}] | [{'20190101': '11 MG road, chennai 600021'}, {'20200101': '921 gandhi road, bangalore 600021'}]   | bangalore | female   |   300000 |


In [33]:
df["marks"] = df["marks"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df["address"] = df["address"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
marks_df = df["marks"].apply(lambda x: {k: v for d in x for k, v in d.items()})
marks_df = pd.DataFrame(marks_df.tolist())
df = df.drop(columns=["marks"]).join(marks_df)
address_list = []
for row in df["address"]:
    for entry in row:
        for date, addr in entry.items():
            address_list.append({"date": date, "address": addr})

address_df = pd.DataFrame(address_list)
df = df.drop(columns=["address"]).merge(address_df, how="left", left_index=True, right_index=True)
pty_print(df)

|    | name   |   age | city      | gender   |   salary |   english |   tamil |   math |   science |     date | address                          |
|----|--------|-------|-----------|----------|----------|-----------|---------|--------|-----------|----------|----------------------------------|
|  0 | Raj    |    21 | chennai   | male     |   200000 |        91 |      99 |     87 |        89 | 20220101 | 331 main road, chennai 600021    |
|  1 | Vim    |    28 | bangalore | female   |   300000 |        93 |      98 |     99 |       100 | 20210101 | 33 gandhi road, bangalore 600021 |


In [34]:
url = 'https://raw.githubusercontent.com/leninworld/parser_exercises/refs/heads/main/json/twitter/twitter.json'
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
else:
    print(f"Failed to fetch data: {response.status_code}")
    exit()
df = pd.json_normalize(data)
pty_print(df)

|    | created_at                     |                 id |             id_str | text                                                                                                            | truncated   | source                                                   | entities.hashtags                            | entities.symbols   | entities.user_mentions   | entities.urls                                                                                                                          |   user.id |   user.id_str | user.name            | user.screen_name   | user.location        | user.description                                                           | user.url               | user.entities.url.urls                                                                                                                                     | user.entities.description.urls   | user.protected   |   user.followers_count |   user.friends_count |   user.listed_count | user.created_at      